# Load and print the .env file and print the OPENAI_API_KEY variable

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

openai_api_key=os.getenv("OPENAI_API_KEY")

print("OPENAI_API_KEY Length:", len(openai_api_key))

OPENAI_API_KEY Length: 164


# Hello world with LangChain and OpenAI's GPT-3.5 Turbo

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")

response=llm.invoke("Hello, world! This is my first call to LangChain using OpenAI's GPT-3.5 Turbo model.")

print(response.content)


Hello! Welcome to LangChain. How can I assist you today?


# Prompt Templates

In [6]:
from langchain_core.prompts import PromptTemplate

template = PromptTemplate(
    input_variables=["topic"],
    template="Tell me a joke about {topic}?",
)

prompt = template.format(topic="chickens")
response = llm.invoke(prompt)
print(response.content)

Why did the chicken join a band? Because it had the drumsticks!


# Chaining Steps

In [7]:
chain = template | llm
response = chain.invoke({"topic": "computers"})
print(response.content)

Why was the computer cold?

It left its Windows open!


# Output Parser

In [8]:
from langchain_core.output_parsers import StrOutputParser

chain = template | llm | StrOutputParser()
response = chain.invoke({"topic": "programming"})
print(response)

Why do programmers prefer dark mode? Because the light attracts bugs!


# Defining and using tools

In [9]:
from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int:
    """multiply two numbers together."""
    return a * b

# create a list of tools
tools = [multiply]

llm_with_tools = llm.bind_tools(tools)

response = llm_with_tools.invoke("What is 6 multiplied by 7?")

print(response)



content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 55, 'total_tokens': 72, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CkoFPEkKxqiqoC8rCHeB1F87079jW', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--019b026d-469b-7e92-8d4a-6b2c1210713d-0' tool_calls=[{'name': 'multiply', 'args': {'a': 6, 'b': 7}, 'id': 'call_yZBUI9BGvO7O1IN3xkcdwn0J', 'type': 'tool_call'}] usage_metadata={'input_tokens': 55, 'output_tokens': 17, 'total_tokens': 72, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [10]:
tool_call = response.tool_calls[0]
arguments = tool_call["args"]
result = multiply.invoke(arguments)
print("Tool Result:", result)

Tool Result: 42


# Agent Executor (or in newer versions, a Graph)

In [11]:
from langchain.agents import create_agent

# We already have 'llm' and 'tools' from before
agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt="You are a helpful assistant who is good at math."
)

In [12]:
# We pass a dictionary with a "messages" key
# The value is a list containing our user message
response = agent.invoke({
    "messages": [{"role": "user", "content": "What is 6 multiplied by 7?"}]
})

# Print the last message in the response (which is the AI's final answer)
print(response["messages"][-1].content)

6 multiplied by 7 is 42.


# Memory

In [19]:
chat_history = []

conversation = {"role": "user", "content": "Hi, my name is Bob."}
chat_history.append(conversation)
question = chat_history + [{"role": "user", "content": "What is my name?"}]
print(f"Question with history: {question}")
response = agent.invoke({
    "messages": question
})
print(response["messages"][-1].content)

chat_history += question
chat_history += [{"role": "assistant", "content": response["messages"][-1].content}]
print(chat_history)



Question with history: [{'role': 'user', 'content': 'Hi, my name is Bob.'}, {'role': 'user', 'content': 'What is my name?'}]
Your name is Bob.
[{'role': 'user', 'content': 'Hi, my name is Bob.'}, {'role': 'user', 'content': 'Hi, my name is Bob.'}, {'role': 'user', 'content': 'What is my name?'}, {'role': 'assistant', 'content': 'Your name is Bob.'}]
